<a href="https://colab.research.google.com/github/satani99/practical_deep_learning_for_coders/blob/main/fast_ai_8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [30]:
from fastai.collab import *
from fastai.tabular.all import *
path = untar_data(URLs.ML_100k)

In [31]:
ratings = pd.read_csv(path/'u.data', delimiter='\t', header=None,
                      names=['user', 'movie', 'rating', 'timestamp'])
ratings.head()

,user,movie,rating,timestamp
0,196,242,3,881250949
1,186,302,3,891717742
2,22,377,1,878887116
3,244,51,2,880606923
4,166,346,1,886397596


In [32]:
last_skywalker = np.array([0.98, 0.9, -0.9])

In [33]:
user1 = np.array([0.9, 0.8, -0.6])

In [34]:
(user1@last_skywalker.T)

2.1420000000000003

In [35]:
casablanca = np.array([-0.99, -0.3, 0.8])

In [36]:
(user1@casablanca.T)

-1.611

In [37]:
movies = pd.read_csv(path/'u.item', delimiter='|', encoding='latin-1',
                     usecols=(0, 1), names=('movie', 'title'), header=None)
movies.head()

,movie,title
0,1,Toy Story (1995)
1,2,GoldenEye (1995)
2,3,Four Rooms (1995)
3,4,Get Shorty (1995)
4,5,Copycat (1995)


In [38]:
ratings = ratings.merge(movies)
ratings.head()

,user,movie,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


In [39]:
dls = CollabDataLoaders.from_df(ratings, item_name='title', bs=64)
dls.show_batch()

,user,title,rating
0,339,Strictly Ballroom (1992),5
1,465,"Clockwork Orange, A (1971)",3
2,621,Son in Law (1993),4
3,293,"Sex, Lies, and Videotape (1989)",3
4,624,Desperate Measures (1998),3
5,566,2001: A Space Odyssey (1968),5
6,486,Willy Wonka and the Chocolate Factory (1971),2
7,896,"Crow, The (1994)",3
8,514,Much Ado About Nothing (1993),5
9,293,Rear Window (1954),5


In [40]:
n_users = len(dls.classes['user'])
n_movies = len(dls.classes['title'])
n_factors = 5

user_factors = torch.randn(n_users, n_factors)
movie_factors = torch.randn(n_movies, n_factors)

In [41]:
one_hot_3 = one_hot(3, n_users).float()
user_factors.t() @ one_hot_3

tensor([ 2.4153, -0.0741,  0.0787,  0.1010, -0.4411])

In [42]:
user_factors[3]

tensor([ 2.4153, -0.0741,  0.0787,  0.1010, -0.4411])

In [43]:
class DotProduct(Module):
  def __init__(self, n_users, n_movies, n_factors, y_range=(0, 5.5)):
    self.user_factors = Embedding(n_users, n_factors)
    self.user_bias = Embedding(n_users, 1)
    self.movie_factors = Embedding(n_movies, n_factors)
    self.movie_bias = Embedding(n_movies, 1)
    self.y_range = y_range

  def forward(self, x):
    users = self.user_factors(x[:, 0])
    movies = self.movie_factors(x[:, 1])
    res = (users * movies).sum(dim=1, keepdim=True)
    res += self.user_bias(x[:,0]) + self.movie_bias(x[:,1])
    return sigmoid_range(res, *self.y_range)

In [44]:
x, y = dls.one_batch()
x.shape

torch.Size([64, 2])

In [45]:
model = DotProduct(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())

In [46]:
learn.fit_one_cycle(5, 5e-3, wd=0.2)

epoch,train_loss,valid_loss,time
0,0.959815,0.949015,00:11
1,0.854711,0.899792,00:12
2,0.813835,0.851070,00:11
3,0.754230,0.820420,00:11
4,0.689595,0.818033,00:10


In [47]:
class T(Module):
  def __init__(self): self.a = nn.Linear(1, 3, bias=False)

t = T()
L(t.parameters())

(#1) [Parameter containing:
tensor([[-0.1197],
        [ 0.5591],
        [ 0.2658]], requires_grad=True)]

In [48]:
type(t.a.weight)

torch.nn.parameter.Parameter

In [49]:
def create_params(size):
  return nn.Parameter(torch.zeros(*size).normal_(0, 0.01))

In [50]:
class DotProductBias(Module):
  def __init__(self, n_users, n_movies, n_factors, y_range=(0,5.5)):
    self.user_factors = create_params([n_users, n_factors])
    self.user_bias = create_params([n_users])
    self.movie_factors = create_params([n_movies, n_factors])
    self.movie_bias = create_params([n_movies])
    self.y_range = y_range

  def forward(self, x):
    users = self.user_factors[x[:,0]]
    movies = self.movie_factors[x[:,1]]
    res = (users*movies).sum(dim=1)
    res += self.user_bias[x[:,0]] + self.movie_bias[x[:,1]]
    return sigmoid_range(res, *self.y_range)

In [51]:
model = DotProductBias(n_users, n_movies, 50)
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.952389,0.940298,00:11
1,0.858611,0.867210,00:11
2,0.705439,0.826913,00:11
3,0.582979,0.814149,00:11
4,0.481528,0.815492,00:11


In [52]:
movie_bias = learn.model.movie_bias.squeeze()
idxs = movie_bias.argsort()[:5]
[dls.classes['title'][i] for i in idxs]

['Children of the Corn: The Gathering (1996)',
 'Robocop 3 (1993)',
 'Barb Wire (1996)',
 "Stephen King's The Langoliers (1995)",
 'Leave It to Beaver (1997)']

In [53]:
idxs = movie_bias.argsort(descending=True)[:5]
[dls.classes['title'][i] for i in idxs]

['Shawshank Redemption, The (1994)',
 'L.A. Confidential (1997)',
 'Titanic (1997)',
 'Apt Pupil (1998)',
 'Silence of the Lambs, The (1991)']

In [54]:
learn = collab_learner(dls, n_factors=50, y_range=(0, 5.5))
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.953739,0.939027,00:11
1,0.882377,0.864779,00:11
2,0.750452,0.824781,00:12
3,0.589982,0.812079,00:11
4,0.489430,0.812937,00:11


In [55]:
learn.model

EmbeddingDotBias(
  (u_weight): Embedding(944, 50)
  (i_weight): Embedding(1665, 50)
  (u_bias): Embedding(944, 1)
  (i_bias): Embedding(1665, 1)
)

In [56]:
movie_bias = learn.model.i_bias.weight.squeeze()
idxs = movie_bias.argsort(descending=True)[:5]
[dls.classes['title'][i] for i in idxs]

['Shawshank Redemption, The (1994)',
 'Titanic (1997)',
 'L.A. Confidential (1997)',
 'Apt Pupil (1998)',
 'Good Will Hunting (1997)']

In [57]:
movie_factors = learn.model.i_weight.weight
idx = dls.classes['title'].o2i['Silence of the Lambs, The (1991)']
distances = nn.CosineSimilarity(dim=1)(movie_factors, movie_factors[idx][None])
idx = distances.argsort(descending=True)[1]
dls.classes['title'][idx]

'Shawshank Redemption, The (1994)'

In [58]:
embs = get_emb_sz(dls)
embs

[(944, 74), (1665, 102)]

In [59]:
class CollabNN(Module):
  def __init__(self, user_sz, item_sz, y_range=(0, 5.5), n_act=100):
    self.user_factors = Embedding(*user_sz)
    self.item_factors = Embedding(*item_sz)
    self.layers = nn.Sequential(
        nn.Linear(user_sz[1]+item_sz[1], n_act),
        nn.ReLU(),
        nn.Linear(n_act, 1)
    )
    self.y_range = y_range

  def forward(self, x):
    embs = self.user_factors(x[:,0]), self.item_factors(x[:, 1])
    x = self.layers(torch.cat(embs, dim=1))
    return sigmoid_range(x, *self.y_range)

    

In [60]:
model = CollabNN(*embs)

In [61]:
learn = Learner(dls, model, loss_func=MSELossFlat())
learn.fit_one_cycle(5, 5e-3, wd=0.01)


epoch,train_loss,valid_loss,time
0,0.918685,0.943393,00:26
1,0.899905,0.928208,00:17
2,0.851885,0.873091,00:14
3,0.811494,0.860996,00:14
4,0.759008,0.865840,00:14


In [62]:
learn = collab_learner(dls, use_nn=True, y_range=(0, 5.5), layers=[100,50])
learn.fit_one_cycle(5, 5e-3, wd=0.1)

epoch,train_loss,valid_loss,time
0,0.974756,1.002898,00:27
1,0.938717,0.914273,00:28
2,0.921935,0.879080,00:21
3,0.810191,0.857179,00:15
4,0.740585,0.860333,00:16
